In [9]:
'''
If you change the csound_module.py, run this cell to reload
reload_module()

'''
from sympy.external import importtools
from csound_module import *
import librosa # type: ignore
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [13]:
audio_folder = 'dataset'
csound = Csound_instrument()
X_train = []  
y_train = []

params_list = [
    # Original 5
    {'freq': 220, 'dur': 1.0, 'amp': 0.5},
    {'freq': 440, 'dur': 1.0, 'amp': 0.5},
    {'freq': 880, 'dur': 0.5, 'amp': 0.5},
    {'freq': 330, 'dur': 1.5, 'amp': 0.5},
    {'freq': 550, 'dur': 0.8, 'amp': 0.5},
    {'freq': 261, 'dur': 0.7, 'amp': 0.5},
    {'freq': 293, 'dur': 1.2, 'amp': 0.5},
    {'freq': 349, 'dur': 0.6, 'amp': 0.5},
    {'freq': 392, 'dur': 1.4, 'amp': 0.5},
    {'freq': 493, 'dur': 0.9, 'amp': 0.5},
    {'freq': 523, 'dur': 1.1, 'amp': 0.5},
    {'freq': 587, 'dur': 0.8, 'amp': 0.5},
    {'freq': 659, 'dur': 1.3, 'amp': 0.5},
    {'freq': 698, 'dur': 0.5, 'amp': 0.5},
    {'freq': 784, 'dur': 1.0, 'amp': 0.5},
    {'freq': 250, 'dur': 1.5, 'amp': 0.5},
    {'freq': 370, 'dur': 0.6, 'amp': 0.5},
    {'freq': 600, 'dur': 1.2, 'amp': 0.5},
    {'freq': 750, 'dur': 0.7, 'amp': 0.5},
    {'freq': 900, 'dur': 0.9, 'amp': 0.5},
]
for i, params in enumerate(params_list):
    filename = f"dataset/train_{i}.wav"
    csound.csound_instrument_score(
        instr_number=1,
        frequency=params['freq'],
        duration=params['dur'],
        save_file=filename
    )
    
    audio, sr = librosa.load(filename, sr=48000)
    if audio.ndim == 1:
        audio = audio[np.newaxis, :]
    
    max_len = 48000 * 2  # 2秒
    if audio.shape[1] < max_len:
        audio = np.pad(audio, ((0, 0), (0, max_len - audio.shape[1])))
    else:
        audio = audio[:, :max_len]
    
    X_train.append(audio)
    y_train.append([params['freq'], params['dur'], params['amp']])

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_train[:, 0] = (y_train[:, 0] - 220) / (900 - 220)  # freq: 220-900 → 0-1
y_train[:, 1] = (y_train[:, 1] - 0.5) / (1.5 - 0.5)  # dur: 0.5-1.5 → 0-1  
y_train[:, 2] = y_train[:, 2] / 1.0                   # amp: already 0-1



rtaudio: PortAudio module enabled ...
using callback interface
rtmidi: PortMIDI module enabled
--Csound version 6.18 (double samples) Nov 24 2022
[commit: a1580f9cdf331c35dceb486f4231871ce0b00266]
libsndfile-1.1.0
graphics suppressed, ascii substituted
sr = 48000.0, kr = 1500.000, ksmps = 32
0dBFS level = 1.0, A4 tuning = 440.0
orch now loaded
audio buffered in 1024 sample-frame blocks
writing 4096-byte blks of shorts to  _temp_1.wav (WAV)
SECTION 1:
new alloc for instr 1:
B  0.000 ..  1.000 T  1.000 TT  1.000 M:  0.50000  0.50000
Score finished in csoundPerformKsmps() with 2.
inactive allocs returned to freespace
end of score.		   overall amps:  0.50000  0.50000
	   overall samples out of range:        0        0
0 errors in performance
Elapsed time at end of performance: real: 0.002s, CPU: 0.002s
2048 4096 sample blks of shorts written to  _temp_1.wav (WAV)
rtaudio: PortAudio module enabled ...
using callback interface
rtmidi: PortMIDI module enabled
--Csound version 6.18 (double sam

In [11]:
train_dataset = torch.utils.data.TensorDataset(X_train, y_train)
train_loader = torch.utils.data.DataLoader(train_dataset)

def get_data():
    return train_loader
